#Import all necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


#Read DataSets


In [ ]:
df = pd.read_csv(r'C:\Users\kevin\Downloads\EEG.machinelearing_data_BRMH.csv')

##Check Attributes

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.describe()

Found typo error in specific disorder(not edited as per current requirements)

#Check Duplicates

In [ ]:
df.duplicated().sum()

#Check Missing Values

In [ ]:
missing_values = df.isna().sum()
missing_value_columns = missing_values[missing_values > 0]
print(missing_value_columns)

Here ID and Education has missing values

#EDA(Exploratory data analysis)

In [ ]:
sns.countplot(x= df.sex)

In [ ]:
sns.countplot(y=df['main.disorder'])

In [ ]:
df.columns

In [ ]:
sns.displot(x=df['IQ'], col=df['main.disorder'], hue=df['sex'])
plt.show()

In [ ]:
sns.displot(x=df['age'], col=df['main.disorder'], hue=df['sex'])
plt.show()

In [ ]:
sns.displot(x=df['education'], col=df['main.disorder'], hue=df['sex'])
plt.show()

In [ ]:
sns.displot(x=df['no.'], col=df['main.disorder'], hue=df['sex'])
plt.show()

This shows age, education and IQ has a relation with mental disorders.
Age also shows a relation(Further check suggested)

In [ ]:
df.columns
#As age 

#Removing Specific disorder as they are not important rn

In [ ]:
df_edited = df.drop('specific.disorder', axis=1)
print(df_edited)

#Making Non numeric feature main.disorder to numeric by using label encoder

In [ ]:
encoder = LabelEncoder()
df_edited['main.disorder'] = encoder.fit_transform(df_edited['main.disorder'])
print(df_edited)

#Converting Sex column to numeric

In [ ]:
df_edited = pd.get_dummies(data=df_edited, columns=['sex'])

#Null Value Treatment

In [ ]:
df_edited.isna().sum()

In [ ]:
imputer = KNNImputer(n_neighbors=2)
df_imputed = imputer.fit_transform(df_edited[['education', 'IQ']])
df_imputed = pd.DataFrame(df_imputed, columns=['education', 'IQ'])
#print(df_imputed)
df_edited[['education', 'IQ']] = df_imputed


##Finding the Unnamed column name

In [ ]:
unname_index = df_edited.columns.get_loc('Unnamed: 122')
print(unname_index)

In [ ]:
print(df_edited.iloc[:,120])
#Name: AB.F.gamma.s.O2
print(df_edited.iloc[:,122])
#Name: COH.A.delta.a.FP1.b.FP2

In [ ]:
df_new = df_edited.drop(columns=['Unnamed: 122'], axis=1)
missing = df_new.isna().sum()
missing_cols = missing[missing > 0]
df_new.shape
#Initial Shape- (945, 1149)
#Final shape- (945, 1147) //dropped specific disorder and unnamed 122

#Preprocessing

##Splitting the features and target

In [ ]:
X = df_new.drop(columns=['no.', 'eeg.date', 'main.disorder'], axis=1)
y = df_new['main.disorder']
X

##Taking logarithmic transformation for scalablity of data except for sex education and IQ

In [ ]:
log_trans_cols = X.drop(['sex_M', 'sex_F', 'education', 'IQ', 'age'], axis=1).columns
X[log_trans_cols] = np.log(X[log_trans_cols])

##Splitting data into train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


##Scaling the features

In [ ]:
scaler = StandardScaler()
X_train_new = scaler.fit_transform(X_train)
X_test_new = scaler.fit_transform(X_test)

#Fitting data to Model

In [ ]:
#Here we use Logistic regression or any other efficient algorithm for label classification

In [ ]:
clf_LG = LogisticRegression(solver='saga', penalty='elasticnet', l1_ratio=0.025, C=0.01).fit(X_train_new, y_train)

In [ ]:
y_pred = clf_LG.predict(X_test_new)


In [ ]:
#y_test_new = y_test.reset_index(drop=True)
comparison = pd.DataFrame({'True Values': y_test, 'Predictions': y_pred})
comparison

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Print accuracy score
print("Accuracy:", accuracy_score(y_test, y_pred))

# Detailed report
print(classification_report(y_test, y_pred))
